In [ ]:
# Initialize Otter Grader
import otter
grader = otter.Notebook()

![data-x](https://raw.githubusercontent.com/afo/data-x-plaksha/master/imgsource/dx_logo.png)

___

#### NAME:

#### STUDENT ID:
___

#  HW3-4: Time Series
**(Total 120 points)**

Run the following cell to install the packages you need for this assignment. You only need to install once, and you can comment the installation cell later.

In [1]:
# Run the cell to install the packages

!python3 -m pip install pystan
!conda install -c conda-forge fbprophet --yes

Run the following cell to load the required modules.

In [2]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

We will now work on an Air Traffic Passenger dataset from https://data.sfgov.org/Transportation/Air-Traffic-Passenger-Statistics/rkru-6vcg. This dataset contains the monthly passenger traffic statistics by airline.

In [3]:
# Read in datasets
flight_stat = pd.read_csv('SF_Air_Traffic_Passenger_Statistics_2005_2020.csv')
display(flight_stat)

We will consider the passenger count vs. time. Therefore, let us sum the passenger count from all different airlines by grouping the date.

In [4]:
flight = flight_stat[['Date', 'Passenger Count']].groupby('Date').sum().reset_index()
flight['Date'] = pd.to_datetime(flight['Date'], format='%Y%m%d', errors='ignore')
flight = flight.set_index('Date')
display(flight)

## 1. Measuring Error with MAPE
**(Total 11 points)**

The Mean Absolute Percentage Error (MAPE) is a common metric for measuring error in forecasts. The MAPE represents error as a percentage of the actual observed values. A high MAPE value means the error is large relative to the quantity being measuring and so the forecast is poor. A small MAPE means the error is relatively small so the forecast is good.

The MAPE is given by:

$$ MAPE = \frac{1}{n} \sum_{i=1}^{n} \frac{|F_t - A_t|}{A_t} $$

where $A_t$ is the actual observed value, $F_t$ is the forecast, and $n$ is the number of data points the MAPE is being calculated over. Read about MAPE at https://en.wikipedia.org/wiki/Mean_absolute_percentage_error

### 1.a MAPE Function

Write a function `get_mape` that calculates the MAPE for a set of forecasts and actuals. The inputs to the function are two <i> pd.series called </i> `actuals` and `forecasts`. The function returns the MAPE as decimal rounded to 5 decimal places (i.e. MAPE = 0.10325)

<!--
BEGIN QUESTION
name: q1a
manual: true
points: 11
-->
<!-- EXPORT TO PDF -->

In [5]:
def get_mape(actuals, forecasts):
    MAPE = ...
    return MAPE

In [ ]:
grader.check("q1a")

## 2. Selecting Parameters in Exponential Smoothing
**(Total 36 points)**

Plot the monthly flight passengers dataset to understand the trend and seasonality. Does the trend appear to be multiplicative or additive? Does the seasonality appear to be multiplicative or additive? You can read different types of trend & seasonality in https://datax.berkeley.edu/wp-content/uploads/2020/09/slides-m215-time-series.pdf.

In [8]:
plt.title('Monthly Flight Passengers in SFO Airport')
plt.ylabel('Passenger Count')
plt.plot(flight['Passenger Count']);

For simplicity, let us only consider the data before the pandemic, that is, February 1st, 2020. We will use the data before 2017 as the training data, and after that as test data.

In [9]:
flight = flight[flight.index < '2020-02-01']
train = flight[flight.index <= '2016-12-31']
test = flight[flight.index >= '2017-01-01']

### 2.a Triple Exponential Smoothing (Holt-Winters Seasonal)

Use Triple Exponential Smoothing to create monthly flight passenger count forecasts for Jan 2017 and onward (37 months into the future). You can read more about exponential smoothing [here](https://machinelearningmastery.com/exponential-smoothing-for-time-series-forecasting-in-python). Fit a model with parameter values recommended by the model and train on data collected from Dec 2016 and prior.
##### `ExponentialSmoothing(...)`
- `trend` = [FROM PART A]: {"additive", "multiplicative", None} 
    Type of trend component.
- `seasonal` = [FROM PART A]: {"additive", "multiplicative", None}
    Type of seasonal component.

- `seasonal_periods` = ?: The number of periods in a complete seasonal cycle, e.g., 4 for quarterly data or 7 for daily data with a weekly cycle.



##### `ExponentialSmoothing().fit(...)`
- `smoothing_level` = Recommended by model 

    Parameter (also called alpha) between 0 and 1 that determines how to weigh previous observations. A value closer to 1 means the model pays more attention to recent observations and a value closer to 0 tells the model to use older observations.
- `smoothing_slope`  = Recommended by model

    Parameter (also called beta) between 0 and 1 that controls the trend. If alpha is closer to 1, the model pays attention to the more recent trend. 
- `smoothing_seasonal`  = Recommended by model

    Parameter (also called gamma) between 0 and 1 that controls the seasonality. If gamma is closer to 1, the model pays attention to more recent seasonality cycle's shape. 


Hint: Remember we can let the model recommend parameters by passing no inputs into the `fit` method:  `ExponentialSmoothing(...).fit()`


<!--
BEGIN QUESTION
name: q2a
manual: true
points: 12
-->
<!-- EXPORT TO PDF -->

In [10]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

### YOUR CODE HERE
model = ...

# Fit models
fit = ...

# Forecast 
pred = ...


plt.plot(pred, label = 'alpha, beta, gamma = recommended')
plt.plot(flight['Passenger Count'])
plt.legend();

In [ ]:
grader.check("q2a")

### 2.b Parameter Selection
Now use Triple Exponential Smoothing to create monthly flight passenger count forecasts for Jan 2017 and onward (37 months into the future) but use the following model parameters. Again fit models on the training data collected from Dec 2016 and prior. Store the new fits in `fit1` and `fit2` and new prediction in `pred1` and `pred2`. Is the forecast better when the model recommends parameters or with these values?


Use Triple Exponential Smoothing 

- `trend` = [FROM PART A]
- `seasonal` = [FROM PART A]
- `seasonal_periods` = ?
- `smoothing_level` = 0.0001 for pred1, 0.1 for pred2
- `smoothing_slope` = 0.0001 for pred1, 0.1 for pred2
- `smoothing_seasonal` = 0.0001 for pred1, 0.1 for pred2


<!--
BEGIN QUESTION
name: q2b
manual: true
points: 12
-->
<!-- EXPORT TO PDF -->

In [17]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing


### YOUR CODE HERE

# Fit models
fit1 = ...
fit2 = ...

# Forecast 
pred1 = ...
pred2 = ...


plt.plot(pred, label = 'alpha, beta, gamma = recommended')
plt.plot(pred1, label = 'alpha, beta, gamma = 0.0001')
plt.plot(pred2, label = 'alpha, beta, gamma = 0.1')
plt.plot(flight['Passenger Count'])
plt.legend();

In [ ]:
grader.check("q2b")

### 2.c Sweep the Parameter
Calculate the MAPE as the `smoothing_slope` parameter changes from 0.01 to 1 in intervals of 0.01. Other two parameters remain `0.1`. Train your model on all data from Dec 2016 and prior. Calculate the MAPE by comparing the 37 months of forecasts to the test set (Jan 2017 and onward). Record the MAPE values in a list called `mapes` where the first element is calculated with beta = 0.01 and the last value is calculated with beta = 1.

<!--
BEGIN QUESTION
name: q2c
manual: true
points: 12
-->
<!-- EXPORT TO PDF -->

In [21]:
### YOUR CODE HERE ###

def score_train_model(model, beta):
    # Fit model
    fit = ...

    # Forecast
    pred = ...
    return get_mape(test['Passenger Count'], pred)


model = ...

mapes = ...
betas = ...
for b in betas:
    score = ...
    mapes.append(score)

In [ ]:
grader.check("q2c")

We'll plot the error below. We should see that the error is minimized when beta is between about 0.6 and 0.8. A similar searching method can be used to select the other parameter values.

In [25]:
#Plot MAPE against Betas
plt.plot(betas, mapes)
plt.xlabel('Betas')
plt.ylabel('MAPE')
plt.title('Mean Absolute Percentage Error (MAPE) vs. Betas');

## 3. Comparing ARIMA and SARIMA Models
**(Total 36 points)**

### 3.a ARIMA Model

[ARIMA](https://machinelearningmastery.com/arima-for-time-series-forecasting-with-python/) stands for AutoRegressive Integrated Moving Average. The ARIMA model first removes autocorrelation in the time series and then uses a regression model to make a a forecast off of recent observations and forecast error.

- `order`: a set of parameters `(p, d, q)` specifing the order of the autoregressive, moving average, and differencing terms.
    - `p` : The number of past observations to use or autogressive order. (If you are forecasting for week 5 and p = 2, the model would use the week 3 and 4 time series values.)
    - `d` : The order of differencing. The ARIMA model works best if the time series values are not correlated. This autocorrelation can be removed by taking each observation and subtracting the previous observation. (Subtract the week 1 value from week 2, subtract week 2 from week 3 and so on to create a new time series. If the new time series is still autocorrelated, we can repeat the differencing process again.)
    - `q` : The order of the moving average term. The ARIMA model uses the error between the value the forecast model would have predicted and the actual observed value for past time series values. We use q to specify the number of past error terms to include in the model.

Use an ARIMA model to forecast monthly flight passenger counts for Jan 2017 and onward (37 months into the future). Train the model on the data from Dec 2016 and prior. Use the following  parameters. Then plot the ARIMA forecast, test set, and training set.

The `arima_pred` will be a tuple, and `arima_pred[0]` will be used to plot the forecast.



- Autoregressive order = 1
- Moving average order = 1
- Differencing order = 1

<!--
BEGIN QUESTION
name: q3a
manual: true
points: 12
-->
<!-- EXPORT TO PDF -->

In [26]:
from statsmodels.tsa.arima_model import ARIMA

### YOUR CODE HERE ###

# Define model
model_arima = ...

# Fit model
model_arima_fit = ...

# Create forecasts
arima_pred = ...

# Plot forecast, test set, and training set
...
...

In [ ]:
grader.check("q3a")

### 3.b SARIMA Model

[SARIMA](https://machinelearningmastery.com/sarima-for-time-series-forecasting-in-python/) stands for Seasonal AutoRegressive Integrated Moving Average. The SARIMA model is the same as the ARIMA model, however, it accounts for seasonality by looking at the corresponding observations across cycles. For example to forecast next December, the SARIMA model might use the value from the last few Decembers, perform differencing over past December observations, and use historical December errors. 

- `order`: a set of parameters `(p, d, q)` specifing the order of the autoregressive, moving average, and differencing terms. Same as the ARIMA model.
- `seasonal_order`: a set of parameters `(P, D, Q, m)` for the seasonal cycles. The parameters (P,D,Q) have the same meaning as in the ARIMA models but for the seasonal components. 
    - `m` : Number of periods in a complete seasonal cycle. (Called seasonal_periods in triple exponential smoothing)

Use a SARIMA model to forecast monthly flight passenger counts for Jan 2017 and onward (37 months into the future). Train the model on the data from Dec 2016 and prior. Use the following parameters. Then plot the ARIMA forecast, test set, and training set.


- Differencing order = 1
- Autoregressive order = 1
- Moving average order = 1
- Seasonal differencing order = 1
- Seasonal autoregressive order = 1
- Seasonal moving average order = 1

<!--
BEGIN QUESTION
name: q3b
manual: true
points: 12
-->
<!-- EXPORT TO PDF -->

In [29]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

### YOUR CODE HERE ###
# Define model
model_sarimax = ...

# Fit model
model_sarimax_fit = ...

# Create forecasts
sarima_pred = ...


# Plot forecast, test set, and training set
plt.plot(sarima_pred, c = 'r')
plt.plot(flight['Passenger Count'], c = 'b')

In [ ]:
grader.check("q3b")

### 3.c ARIMA and SARIMA MAPE Comparison
C. Calculate the MAPE of the ARIMA and SARIMA forecasts by comparing the 37 months of forecasts to the test set.

<!--
BEGIN QUESTION
name: q3c
manual: true
points: 12
-->

<!-- EXPORT TO PDF -->

In [32]:
### YOUR CODE HERE ###

arima_mape = ...
sarima_mape = ...

print('ARIMA MAPE: ', arima_mape)
print('SARIMA MAPE: ', sarima_mape)

In [ ]:
grader.check("q3c")

## 4. Holiday Effects with Facebook's Prophet library

**(Total 37 points)**

Prophet is open source library for time series forecasting developed by Facebook. Prophet builds a model composed of a trend seasonality, and holiday components by fitting a curve through the time series data points.

Unique features include:
- Adding holiday effects (Superbowl, Christmas, New Years, 4th of July)
- Specify points where the trend change
- Adust seasonality smoothness
- Robust to missing values

"Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well." - https://github.com/facebook/prophet


Now we will use a new dataset that contains the total number of users on the [Steam platform](https://steamdb.info/app/753/graphs/) since 2004, daily periodicity. However, the dataset has lots of missing values in the first several years. Let's first read the dataset.

In [37]:
steam_stat = pd.read_csv('Steam_Players.csv')
steam_stat['Date'] = pd.to_datetime(steam_stat['DateTime'])
steam = steam_stat[['Date','Users']].set_index('Date')
display(steam)

Plot the number of users to understand the trend and seasonality.

In [38]:
plt.title('Steam Platform Users')
plt.ylabel('Users')
plt.plot(steam['Users']);

Similarly, let us only consider the data before the pandemic, that is, February 1st, 2020. We will use the data before 2017 as the training data, and after that as test data. 

In [39]:
steam = steam[steam.index < '2020-02-01']
steam_train = steam[steam.index <= '2016-12-31']
steam_test = steam[steam.index >= '2017-01-01']
print(steam.shape, steam_train.shape, steam_test.shape)

### 4.a Prophet Basics
Use Prophet to forecast steam users for 01/01/2017 and onward (1126 days into the future). Train the model on the data from 12/31/2016 and before. Use Prophet's default parameters. Report the forecasts as `prophet_forecast`, a dataframe with 5863 forecasts (including forecasts of training data) where the column 'ds' stores the dates and the column 'yhat' stores the forecast values.

<!--
BEGIN QUESTION
name: q4a
manual: true
points: 12
-->
<!-- EXPORT TO PDF -->

In [40]:
from fbprophet import Prophet

# Prophet requires the time series to be a 2 column data series with the Date as 'ds' and the values as 'y'.

steam_prophet = ...


# Fit the model on the time series.
m_prophet = ...
...

# Create a DataFrame of future dates to create forecasts for. 
future_prophet = ...

# Create forecast
prophet_forecast = ...

fig1 = m_prophet.plot(prophet_forecast)
plt.plot(steam['Users'], c = 'y')

In [ ]:
grader.check("q4a")

We can use the prophet_plot_components function to see the forecast broken down into trend, weekly seasonality, and yearly seasonality.

In [44]:
fig2 = m_prophet.plot_components(prophet_forecast)

### 4.b Add Holiday Impact
Use Prophet to forecast steam users again for 01/01/2017 and onward (1126 days into the future), but now add the Steam's yearly major sale: Winter Sale to the model. The Winter Sale dates are: 2007-12-24, 2008-12-26, 2009-12-22, 2010-12-20, 2011-12-19, 2012-12-20, 2013-12-19, 2014-12-18, 2015-12-22, 2016-12-22,  2017-12-21, 2018-12-20, and 2019-12-19. In average, the Winter Sale lasts for 10 days.

We might also expect that user numbers are affected by holidays. Prophet allows us to add all US Holidays at once to the model.


Train the model on the data from 12/31/2016 and before, and add US holidays and Steam Winter Sales into your model. Use Prophet's default parameters for all other model features. Report the new forecasts as `prophet_forecast_holidays`, a dataframe with 5863 forecasts (including forecasts of training data) where the column 'ds' stores the dates and the column 'yhat' stores the forecast values.


<!--
BEGIN QUESTION
name: q4b
manual: true
points: 15
-->
<!-- EXPORT TO PDF -->

In [45]:
### YOUR CODE HERE ###

holidays = ...
  ...
  ...
  ...
  ...
...


m_prophet_holidays = ...

...

...

future_holidays = ...

prophet_forecast_holidays = ...

fig2 = m_prophet_holidays.plot(prophet_forecast_holidays) 
plt.plot(steam['Users'], c = 'y')

In [ ]:
grader.check("q4b")

### 4.c Compare MAPE
Calculate the MAPE of the Prophet model before account for holidays and after adding holidays. The MAPE should be calculated by comparing the 1126 days of forecasts to the test set.

<!--
BEGIN QUESTION
name: q4c
manual: true
points: 10
-->
<!-- EXPORT TO PDF -->

In [50]:
prophet_forecast = prophet_forecast.reset_index().rename(columns = {'ds':'Date', 'yhat':'Users'})
prophet_forecast = prophet_forecast.set_index('Date')
prophet_forecast_holidays=prophet_forecast_holidays.reset_index().rename(columns = {'ds':'Date', 'yhat':'Users'})
prophet_forecast_holidays = prophet_forecast_holidays.set_index('Date')

prophet_mape = ...
prophet_holiday_mape = ...

print('Original Prophet MAPE: ', prophet_mape)
print('Prophet MAPE with Holidays: ', prophet_holiday_mape)


In [ ]:
grader.check("q4c")

# Submit
Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output.
**Please save before submitting!**

<!-- EXPECT 10 EXPORTED QUESTIONS -->